# Task
Find the actual component values of the bandpass filter in fig. 1 by comparing them with their nominal values.

- Compare the measured response voltage $U_a$ with the expected response voltage of the nominal component values and determine the error. 
- Minimize the calculated error for different component values.
![Circuit Diagramm](circuit.png)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy as sci

# Get Data

In [ ]:
# import measured data from file and create DataFrame
with np.load("messwerte.npz") as file:
    u_a = pd.DataFrame({"time" : file["t_mess"], "measured" : file["u_a_mess"]})


def expected_datapoints(coil, capacitor, time_points):
    # return calculated datapoints by creating LTI System with given parameters
    RESISTOR=20
    VOLTAGE = 2

    numerator=[1/(RESISTOR*capacitor), 0]
    denominator=[1, 1/(RESISTOR*capacitor), 1/(coil*capacitor)]
    system=signal.lti(numerator, denominator)

    return VOLTAGE*sci.signal.step(system, T=time_points)[1]

# append dataframe
COIL=9e-3
CAPACITOR=1000e-6
u_a["calculated"] = expected_datapoints(COIL, CAPACITOR, u_a["time"])

# Analysis

## Plot Expected and Measured Data

In [ ]:
plt.plot(u_a["time"], u_a["measured"], label="Messwerte")
plt.plot(u_a["time"], u_a["calculated"], label='Rechenwerte')

plt.title("Sprungantwort $U_a$ des Bandpasses")
plt.xlabel("Zeit in $s$")
plt.ylabel("$U_a$ in $V$")
plt.legend()

plt.show()

## Find sum of squared error

In [ ]:
def sum_squared_error(calculated, measured): 
    # Calculate the sum square error of 2 1D arrays
    return np.round(np.sum(np.square(np.subtract(measured, calculated))) *10**6, 2)

# calculate the root mean square error for the given hardware parameters
sqe = sum_squared_error(u_a["calculated"], u_a["measured"])
print(f"Quadratische Fehlersumme mit Nennwerten: {sqe} e-6")

# Find minimum root mean square error next to given hardware parameter
COIL_LOWER = 7e-3
COIL_UPPER = 12e-3
CAPACITOR_LOWER = 9 * 100e-6
CAPACITOR_UPPER = 16 * 100e-6
STEPS = 50
COIL_LIST = np.linspace(COIL_LOWER, COIL_UPPER, STEPS)
CAPACITOR_LIST = np.linspace(CAPACITOR_LOWER, CAPACITOR_UPPER, STEPS) 

x, y = np.meshgrid(COIL_LIST, CAPACITOR_LIST)

# Z = sum_squared_error(u_a["calculated"], )

# Z = sum_squared_error(calculated, u_a["measured"])
# plt.contourf(x, y, Z)
# plt.show()

# Z_min = sci.optimize.minimize(sum_squared_error, x0=[0.1**3, 0.1**2])

## Find sum of squared error with variable parameters

In [145]:
Z=[[0]*STEPS]*STEPS
for i in x:
    for j in y:
        Z[i][j] = np.random.normal(0, 1)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# Function to minimize
def objective_function(params):
    coil, capacitor = params
    numerator = [1 / (RESISTOR * capacitor), 0]
    denominator = [1, 1 / (RESISTOR * capacitor), 1 / (coil * capacitor)]
    system = signal.lti(numerator, denominator)
    calculated = VOLTAGE * signal.step(system)[1]
    return mean_root_error(calculated, u_a["measured"])

# Find the minimum RMSE
initial_guess = [0.1**3, 0.1**2]  # Initial guess for COIL and CAPACITOR
result = optimize.minimize(objective_function, x0=initial_guess)
best_coil, best_capacitor = result.x
min_rmse = result.fun

print(f"Optimal COIL: {best_coil}, Optimal CAPACITOR: {best_capacitor}, Minimum RMSE: {min_rmse}")

# Plot the contour of the RMSE values
Z = np.zeros_like(x)
for i in range(STEPS):
    for j in range(STEPS):
        Z[i, j] = mean_root_error(VOLTAGE * signal.step(signal.lti([1 / (RESISTOR * CAPACITOR), 0], [1, 1 / (RESISTOR * CAPACITOR), 1 / (COIL_LIST[i] * CAPACITOR_LIST[j])])[1])[1], u_a["measured"])

plt.contourf(x, y, Z, levels=50, cmap='viridis')
plt.xlabel('COIL')
plt.ylabel('CAPACITOR')
plt.colorbar(label='RMSE')
plt.title('RMSE Contour Plot')
plt.show()



# plt.contourf(x, y, Z)

# ax = plt.figure().add_subplot(projection='3d')
# ax.contour(x, y, Z, extend3d=True)
